In [1]:
# https://www.kaggle.com/franckepeixoto/dogs-x-cats-classifier-using-inceptionv3 
#Goal: Modify the code to where it detects maks and no mask.
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import load_img,img_to_array
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import time
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import shutil
from keras.preprocessing.image import ImageDataGenerator

In [2]:
image_width,image_height = 150,150#299,299
labels =['Mask','No Mask']
for d in labels:
  dir_path = 'Merge_Data/Train/' + d
  if not os.path.exists(dir_path):
    print('{} created.'.format(dir_path))
    os.mkdir(dir_path)
  else:
    print('{} already exists'.format(dir_path))


train_path ="Merge_Data/Train/"
for  file in  os.listdir(train_path):
  category = file.split(".")[0]
  if '.jpg' in file:
    if 'dog'in category: 
      shutil.copyfile(train_path+file,'./Train/Mask/'+ file)
    elif 'cat'in category:  
      shutil.copyfile(train_path+file,'./Train/No Mask/'+ file)

Merge_Data/Train/Mask already exists
Merge_Data/Train/No Mask already exists


In [3]:
print("Total Mask images:\t{}".format(sum([len(files) for r, d, files in os.walk('Merge_Data/Train/Mask')])))
print("Total No Mask images:\t{}".format(sum([len(files) for r, d, files in os.walk('Merge_Data/Train/No Mask')])))

Total Mask images:	4482
Total No Mask images:	4272


In [4]:
keras.backend.clear_session()
batch_size=64
validation_split=0.3
val_size = 7500
dataset_size = 17500 
train_data_generator = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

train_datagenerator = train_data_generator.flow_from_directory(train_path,
                                                     target_size=(image_width,image_height),
                                                     class_mode="categorical",
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     subset='training')
#Figure out what subset mean
val_datagenerator = train_data_generator.flow_from_directory('Merge_Data/Validation',
                                                     target_size=(image_width,image_height),
                                                     class_mode="categorical",
                                                     shuffle=True,
                                                     batch_size=batch_size,
                                                     subset='validation')

Found 8754 images belonging to 4 classes.
Found 0 images belonging to 2 classes.


In [5]:
#Started at the Inceptionv3 model
inception_v3_model = keras.applications.InceptionV3(include_top=False, weights='imagenet',input_shape=(image_width,image_height,3))

In [6]:
keras.backend.clear_session()
x = inception_v3_model.output
avg_pool2d = keras.layers.GlobalAveragePooling2D()(x)
dense = keras.layers.Dense(512, activation= keras.activations.relu)(avg_pool2d)
output = keras.layers.Dense(2,activation=keras.activations.softmax)(dense)
model = keras.Model(inputs=inception_v3_model.input, outputs=output,name = "transfer_inception_v3")

In [7]:
freeze= np.round((len(model.layers)-len(model.layers)*0.3),0).astype('int') 
for layer in model.layers[:freeze]:
    layer.trainable =False
for layer in model.layers[freeze:]:
    layer.trainable=True
model.summary()

Model: "transfer_inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________

In [8]:
plateau_callback = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=2, factor=.5, min_lr=.00001)

start = time.time()

model.compile(optimizer = "Adam", 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

history = model.fit(train_datagenerator,
                  steps_per_epoch=(dataset_size//batch_size),
                  epochs= 5, 
                  verbose=1,
                  validation_data=val_datagenerator,
                  validation_steps=(val_size//batch_size),
                  callbacks=[plateau_callback]
)                                              

Epoch 1/5


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node transfer_inception_v3/conv2d/Conv2D (defined at <ipython-input-8-a133188a3fe1>:9) ]] [Op:__inference_train_function_13418]

Function call stack:
train_function
